<a href="https://colab.research.google.com/github/venkataratnamb20/quicksilicon/blob/notebooks/notebooks/vlsi/pyadc/pyadc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADC


In [1]:
from dataclasses import dataclass

@dataclass
class Adc:
    nbit: int = 10
    vrefp: float = 1.0
    vrefm: float = 0.0
    fs: float = 10e3
    def __post_init__(self, nbit: int = 10, vrefp: float = 1.0, vrefm: float = 0.0, fs: float = 1e3):
        self.nbit = nbit
        self.vrefp = vrefp
        self.vrefm = vrefm
        self.fs = fs

    @property
    def vref(self):
        return self.vrefp - self.vrefm

    @property
    def qlevels(self):
        return 2**self.nbit

    @property
    def lsb(self):
        return self.vref / (self.qlevels - 1)

    @property
    def sqnref(self):
        return 6.02 * self.nbit + 1.76

    @property
    def fclk(self):
        return self.fs * (self.nbit + 2)

    @property
    def tsample(self):
        return 1 / self.fs

    def convert(self, vin: float = 0.0):
        if vin > self.vrefp:
            vin = self.vref
        elif vin < self.vrefm:
            vin = self.vrefm
        return round(vin / self.lsb)

    def sar(self, vin: float = 0.0):
        raise NotImplementedError("Wait for the best tohappen!!!")

    def __str__(self):
        return f"""
        ADC(nbit = {self.nbit}, vrefp = {self.vrefp}, vrefm = {self.vrefm}, fs = {self.fs})"""

    def __repr__(self):
        return self.__str__()

    def __call__(self, vin: float = 0.0):
        return self.convert(vin)


adc = Adc(nbit = 10, vrefp = 1.0, vrefm = 0.0, fs = 10e3)
print(adc)







        ADC(nbit = 10, vrefp = 1.0, vrefm = 0.0, fs = 1000.0)


In [2]:
nbit = 10

for i in range(nbit + 2):
    vin = (2**i) * adc.lsb
    print(f"{vin} -> {adc(vin)}")

0.0009775171065493646 -> 1
0.0019550342130987292 -> 2
0.0039100684261974585 -> 4
0.007820136852394917 -> 8
0.015640273704789834 -> 16
0.03128054740957967 -> 32
0.06256109481915934 -> 64
0.12512218963831867 -> 128
0.25024437927663734 -> 256
0.5004887585532747 -> 512
1.0009775171065494 -> 1023
2.0019550342130987 -> 1023


In [10]:
class SarAdc(Adc):
    def __init__(self, nbit: int = 10, vrefp: float = 0.0, vrefm: float = 0.0, fs: float = 10e3, *args, **kwargs):
        super().__init__(nbit, vrefp, vrefm, fs, *args, **kwargs)
        self.nbit = nbit
        self.vrefp = vrefp
        self.vrefm = vrefm
        self.fs = fs
        self. low_level = self.vrefm
        self.high_level = self.vrefp
        self.code = []

    def comparator(self, vin: float = 0.0, vthresh: float = 0.0):
        return vin > vthresh

    def sar(self, vin: float = 0, *args, **kwargs):
        """Successive Approximation algorithm

        Args:
            vin (float, optional): input voltage. Defaults to 0.
            lower_level (float): lower level voltage. Defaults to 0.
            upper_level (float): upper level voltage. Defaults to 0.

        Returns:
            int: output voltage.
        """
        # nbit = kwargs.get("nbit", 10)
        # vrefm, vrefp = kwargs.get("vrefm", self.vrefm), kwargs.get("vrefp", self.vrefp)
        low_level, high_level = self.vrefm, self.vrefp
        code = []
        def algorithm(low_level: float = self.vrefm, high_level: float  =self.vrefp):
            # vthresh = low_level + (high_level - low_level) / 2
            vthresh = (high_level - low_level) / 2
            # print(f"levels:: low: {low_level} high: {high_level}")
            # print(f"vthresh: {vthresh}")
            decision = int(vin > low_level + vthresh)
            # decision = vin >= vthresh
            code.append(decision)
            if not decision:
                high_level = high_level - vthresh
            else:
                low_level = low_level + vthresh
            if len(code) == self.nbit:
                return code
            else:
                return algorithm(low_level, high_level)
        return algorithm(low_level, high_level)

    def __call__(self, vin: float = 0.0):
        self.code = []
        return self.sar(vin, self.vrefm, self.vrefp)


if __name__ == "__main__":
    saradc = SarAdc()
    print(f"nbit: {saradc.nbit}:: lsb: {saradc.lsb}")
    print(saradc(16 * adc.lsb))
    nbit = 10
    get_analog_from_bits_array = lambda code_list: int(''.join([str(bit) for bit in code_list]), 2)

    for i in range(nbit + 2):
        vin = (2**i) * adc.lsb
        print(f"{2**i}: {vin} \t->\t {saradc(vin)}::{get_analog_from_bits_array(saradc(vin))}")

nbit: 10:: lsb: 0.0
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1: 0.0009775171065493646 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
2: 0.0019550342130987292 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
4: 0.0039100684261974585 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
8: 0.007820136852394917 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
16: 0.015640273704789834 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
32: 0.03128054740957967 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
64: 0.06256109481915934 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
128: 0.12512218963831867 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
256: 0.25024437927663734 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
512: 0.5004887585532747 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
1024: 1.0009775171065494 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
2048: 2.0019550342130987 	->	 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023


## SAR

### SAR: Logic


In [11]:
def sar(vin, *args, **kwargs):
    """Successive Approximation algorithm

    """
    nbit = kwargs.get("nbit", 10)
    vrefm, vrefp = kwargs.get("vrefm", 0), kwargs.get("vrefp", 1.0)
    low_level, high_level = vrefm, vrefp
    lsb = (vrefp - vrefm) / (2**nbit - 1)
    code = []
    def algorithm(vin, low_level, high_level):
        # vthresh = low_level + (high_level - low_level) / 2
        vthresh = (high_level - low_level) / 2
        # print(f"levels:: low: {low_level} high: {high_level}")
        # print(f"vthresh: {vthresh}")
        decision = int(vin > low_level + vthresh)
        # decision = vin >= vthresh
        code.append(decision)
        if not decision:
            high_level = high_level - vthresh
        else:
            low_level = low_level + vthresh
        if len(code) == nbit:
            return code
        else:
            return algorithm(vin, low_level, high_level)
    return algorithm(vin, low_level, high_level)

sar(0.75)



[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [12]:
nbit = 10
get_analog_from_bits_array = lambda code_list: int(''.join([str(bit) for bit in code_list]), 2)

for i in range(nbit + 2):
    vin = (2**i) * adc.lsb
    # print(f"{vin} -> {sar(vin)}::{int(''.join([str(bit) for bit in sar(vin)]), 2)}")
    print(f"{2**i}: {vin} -> {sar(vin)}::{get_analog_from_bits_array(sar(vin))}")

1: 0.0009775171065493646 -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]::1
2: 0.0019550342130987292 -> [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]::2
4: 0.0039100684261974585 -> [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]::4
8: 0.007820136852394917 -> [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]::8
16: 0.015640273704789834 -> [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]::16
32: 0.03128054740957967 -> [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]::32
64: 0.06256109481915934 -> [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]::64
128: 0.12512218963831867 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]::128
256: 0.25024437927663734 -> [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]::256
512: 0.5004887585532747 -> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]::512
1024: 1.0009775171065494 -> [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023
2048: 2.0019550342130987 -> [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]::1023


In [13]:
# int("110", 2)
# int(''.join(sar(0)), 2)
# ''.join([1, 1, 0])

### SAR: test


In [14]:
import unittest

class TestSar(unittest.TestCase):
    def setUp(self):
        self.nbit = 10
        self.vrefp = 1.0
        self.vrefm = 0.0
        self.vref = self.vrefp - self.vrefm
        self.qlevels = 2**self.nbit
        self.lsb = self.vref / (self.qlevels - 1)


        self.get_analog_from_bits_array = lambda code_list: int(''.join([str(bit) for bit in code_list]), 2)

    def test_fullscale_code(self):
        self.assertAlmostEqual(self.get_analog_from_bits_array(sar(self.vrefp)), self.qlevels - 1)

    def test_higher_than_fullscale_code(self):
        self.assertAlmostEqual(self.get_analog_from_bits_array(sar(2 * self.vrefp)), self.qlevels - 1)

    def test_negative_vin(self):
        self.assertAlmostEqual(self.get_analog_from_bits_array(sar(-10 * self.lsb)), 0)

    def test_bit_levels(self):
        for i in range(self.nbit):
            vin = (2**i) * self.lsb
            self.assertAlmostEqual(self.get_analog_from_bits_array(sar(vin)), 2**i)

    def test_all_levels(self):
        for i in range(2**nbit):
            vin = i * self.lsb
            # print(f"{vin} -> {sar(vin)}")
            self.assertAlmostEqual(self.get_analog_from_bits_array(sar(vin)), i)



if __name__ == "__main__":
    unittest.main(argv=[''], exit=False, verbosity=2)

test_all_levels (__main__.TestSar) ... ok
test_bit_levels (__main__.TestSar) ... ok
test_fullscale_code (__main__.TestSar) ... ok
test_higher_than_fullscale_code (__main__.TestSar) ... ok
test_negative_vin (__main__.TestSar) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.034s

OK


In [15]:
print(get_analog_from_bits_array(sar(2**0 * adc.lsb)))
assert get_analog_from_bits_array(sar(2**0 * adc.lsb)) == 2**0
range(2**nbit)[-1]

1


1023

In [16]:
code = "".join([str(bit) for bit in sar(2**0 * adc.lsb)])
assert int(code, 2) == 2**0